In [3]:
from aocd import get_data

puzzle_input = get_data(day=15, year=2015)

In [4]:
print(puzzle_input)

Frosting: capacity 4, durability -2, flavor 0, texture 0, calories 5
Candy: capacity 0, durability 5, flavor -1, texture 0, calories 8
Butterscotch: capacity -1, durability 0, flavor 5, texture 0, calories 6
Sugar: capacity 0, durability 0, flavor -2, texture 2, calories 1


In [1]:
example = """
Butterscotch: capacity -1, durability -2, flavor 6, texture 3, calories 8
Cinnamon: capacity 2, durability 3, flavor -2, texture -1, calories 3
""".strip()

import re

def get_ingredients(text):
    return [
        list(map(int, re.findall(r"-?\d+", s)))
        for s in text.strip().split('\n')
    ]

ingredients_example = get_ingredients(example)

Partie 1

In [6]:
from ortools.sat.python import cp_model

model = cp_model.CpModel()

ingredients = get_ingredients(puzzle_input)
coefficients = list(zip(*ingredients))

alphas = []
for i in range(len(ingredients)):
    alphas.append(model.NewIntVar(0, 100, f"{i}"))

# Somme des alphas doit être égal à 1
model.Add(cp_model.LinearExpr.Sum(alphas) == 100)

for coeffs in coefficients:
    model.Add(cp_model.LinearExpr.WeightedSum(alphas, coeffs) >= 0)

def evaluate_solution(solution, coefficients):
    score = 1

    for coeffs in coefficients:
        score *= max(0, sum(a * c for a, c in zip(solution, coeffs)))

    return score
    
class ObjectiveSolutionPrinter(cp_model.CpSolverSolutionCallback):
    """Display the objective value and time of intermediate solutions."""

    def __init__(self, variables, coefficients):
        cp_model.CpSolverSolutionCallback.__init__(self)
        self.variables = variables
        self.coefficients = coefficients

        self.best_solution = None
        self.best_solution_score = 0

    def on_solution_callback(self):
        """Called on each new solution."""
        score = 1

        solution = tuple(self.Value(v) for v in self.variables)
        solution_score = evaluate_solution(solution, self.coefficients)

        if solution_score > self.best_solution_score:
            self.best_solution = solution
            self.best_solution_score = solution_score

solution_printer = ObjectiveSolutionPrinter(alphas, coefficients[:-1])

solver = cp_model.CpSolver()
solver.parameters.enumerate_all_solutions = True
status = solver.Solve(model, solution_printer)

(solution_printer.best_solution, solution_printer.best_solution_score)

((24, 29, 31, 16), 18965440)

Partie 2

In [7]:
from ortools.sat.python import cp_model

model = cp_model.CpModel()

ingredients = get_ingredients(puzzle_input)
coefficients = list(zip(*ingredients))

alphas = []
for i in range(len(ingredients)):
    alphas.append(model.NewIntVar(0, 100, f"{i}"))

# Somme des alphas doit être égal à 1
model.Add(cp_model.LinearExpr.Sum(alphas) == 100)

# Pour faire le meilleur score il ne faut pas avoir une propriété négative
for coeffs in coefficients[:-1]:
    model.Add(cp_model.LinearExpr.WeightedSum(alphas, coeffs) >= 0)

# Les calories doivent valoir 500
calories_coeffs = coefficients[-1]
wanted_calories = 500
model.Add(cp_model.LinearExpr.WeightedSum(alphas, calories_coeffs) == 500)


def evaluate_solution(solution, coefficients):
    score = 1

    for coeffs in coefficients:
        score *= max(0, sum(a * c for a, c in zip(solution, coeffs)))

    return score
    
class ObjectiveSolutionPrinter(cp_model.CpSolverSolutionCallback):
    """Display the objective value and time of intermediate solutions."""

    def __init__(self, variables, coefficients):
        cp_model.CpSolverSolutionCallback.__init__(self)
        self.variables = variables
        self.coefficients = coefficients

        self.best_solution = None
        self.best_solution_score = 0

    def on_solution_callback(self):
        """Called on each new solution."""
        score = 1

        solution = tuple(self.Value(v) for v in self.variables)
        solution_score = evaluate_solution(solution, self.coefficients)

        if solution_score > self.best_solution_score:
            self.best_solution = solution
            self.best_solution_score = solution_score

solution_printer = ObjectiveSolutionPrinter(alphas, coefficients[:-1])

solver = cp_model.CpSolver()
solver.parameters.enumerate_all_solutions = True
status = solver.Solve(model, solution_printer)

(solution_printer.best_solution, solution_printer.best_solution_score)

((21, 23, 31, 25), 15862900)